##### Copyright 2025 Kensho Technologies, LLC

# Google Gemini Function Calling
**_Gemini to retrieve data from the LLM-ready API using the kFinance python library!_**

What you'll need to run this notebook:

1.   kFinance credentials
2.   A Google Gemini API key

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/kensho-technologies/llm-ready-api-examples/blob/main/function_calling/Google_Gemini_function_calling.ipynb"><img src="../images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

## Install dependencies

In [ ]:
# install the latest version of kFinance package
%pip install https://kfinance.kensho.com/static/kensho_finance.tar.gz
# install the LLM Python package
%pip install google-generativeai

# Instantiate kFinance Client

In [ ]:
# import the kfinance client
from kensho_finance.kfinance import Client
# import standard libraries
import functools
import types
import json
import sys
# check if the current environment is a Google Colab
try:
  import google.colab
  IN_GOOGLE_COLAB = True
except:
  IN_GOOGLE_COLAB = False

# initialize the kfinance client with one of the following:
# 1. your kensho refresh token
# 2. your kensho client id and kensho private key
# 3. automated login (not accessible on Google Collab)
if IN_GOOGLE_COLAB:
    kensho_refresh_token = ""
    assert kensho_refresh_token != "", "kensho refresh token is empty! Make sure to enter your kensho refresh token above"
    kfinance_client = Client(refresh_token=kensho_refresh_token)

    # kensho_client_id = ""
    # kensho_private_key = ""
    # assert kensho_client_id != "", "kensho client id is empty! Make sure to enter your kensho client id above"
    # assert kensho_private_key != "", "kensho private key is empty! Make sure to enter your kensho private key above"
    # kfinance_client = Client(client_id=kensho_client_id, private_key=kensho_private_key)
else:
    kfinance_client = Client()

# the prompt given to all LLMs to instruct them how to use tools to call the kFinance API
SYSTEM_PROMPT = "You are an LLM trying to help financial analysts. Use the supplied tools to assist the user. Always use the `get_latest` function when asked about the last or most recent quarter or time period etc. Always use the `get_latest` function when a tool requires a time parameter and the time is unspecified in the question"
# the message shown to users to prompt them to input a message
USER_INPUT_PROMPT = "Enter your message and press the [return] key\n"

# Google Gemini Function Calling

In [ ]:
# import Google's generativeai package ('Gemini')
import google.generativeai as genai

# the GeminiChat class is used to create a chat loop that automatically executes tool calls
class GeminiChat:
  def __init__(self, kfinance_client):
    # initialize Gemini with your Gemini API key
    gemini_api_key = "" # replace with your own key
    assert gemini_api_key != "", "Gemini API key is empty! Make sure to enter your Gemini API key above"
    # good client config for Gemini. There are other ways to access Gemini
    genai.configure(api_key=gemini_api_key)
    model = genai.GenerativeModel(model_name='gemini-2.0-flash', system_instruction=SYSTEM_PROMPT, tools=kfinance_client.gemini_tool_descriptions)
    self.gemini = model.start_chat()


  def print_responses(self, user_input: str):
      """Print responses and call tools"""
      # send a message to Gemini and get the response
      response = self.gemini.send_message(user_input)
      # parse text and tool calls from the response
      parts = response.candidates[0].content.parts
      text_parts = list(filter(lambda part: "text" in part, parts))
      tool_parts = list(filter(lambda part: "function_call" in part, parts))
      # print text
      for text_part in text_parts:
         sys.stdout.write(text_part.text)
      # while the response has tool calls
      while tool_parts != []:
          tool_outputs = []
          # for each tool call, execute the function and arguments specified in the tool call
          # and append the output as a message to the message history
          for tool_part in tool_parts:
              function = tool_part.function_call.name
              arguments = {}
              # convert arguments to ints if floats, since the tool descriptions schema does not work for ints
              for key, value in tool_part.function_call.args.items():
                arguments[key] = int(value) if isinstance(value, float) else value
              # try to execute the function and arguments specified in the tool call
              # append the output to the list of tool outputs, which will be send to the model
              try:
                output =  str(kfinance_client.tools[function](**arguments))
              except Exception as e:
                output = str(e)
              tool_outputs.append(f"The output of function {function} with arguments {arguments} is: {output}")

          # send the list of tool outputs to the model and get a response
          response = self.gemini.send_message(' '.join(tool_outputs))
          # parse text and tool calls from the response
          parts = response.candidates[0].content.parts
          text_parts = list(filter(lambda part: "text" in part, parts))
          tool_parts = list(filter(lambda part: "function_call" in part, parts))
          # print text
          for text_part in text_parts:
            sys.stdout.write(text_part.text)
      return None

  def start_chatting(self) -> None:
      """Open chat shell"""
      while True:
          user_input = input(USER_INPUT_PROMPT)
          self.print_responses(user_input)
          sys.stdout.write("\n")

In [ ]:
# instantiate the GeminiChat with the kfinance client
gemini_chat = GeminiChat(kfinance_client)
# start chatting with the GeminiChat
gemini_chat.start_chatting()